In [1]:
import pennylane as qml
import torch
from torch.utils.data import DataLoader, TensorDataset
import ray
from ray import tune
from ray.air import session
from sklearn.model_selection import train_test_split
import numpy as np
import yaml
import json
import time
import os
import pandas as pd


# Custom Libraries
from utils.model import Qkernel
from utils.data_generator import DataGenerator
from utils.agent import TrainModel

# Backend Configuration
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

2024-11-14 23:34:02,603	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-14 23:34:02,981	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# Read Configs
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

data = np.load('checkerboard_dataset.npy', allow_pickle=True).item()
x_train, x_test, y_train, y_test = data['x_train'], data['x_test'], data['y_train'], data['y_test']

training_data = torch.tensor(x_train, dtype=torch.float32, requires_grad=True)
testing_data = torch.tensor(x_test, dtype=torch.float32, requires_grad=True)
training_labels = torch.tensor(y_train, dtype=torch.int)
testing_labels = torch.tensor(y_test, dtype=torch.int)

kernel = Qkernel(   
                        device = config['qkernel']['device'], 
                        n_qubits = 4, 
                        trainable = True, 
                        input_scaling = True, 
                        data_reuploading = True, 
                        ansatz = 'he', 
                        ansatz_layers = 5
                    )
    
agent = TrainModel(
                        kernel=kernel,
                        training_data=training_data,
                        training_labels=training_labels,
                        testing_data=testing_data,
                        testing_labels=testing_labels,
                        optimizer= 'gd',
                        lr= 0.1,
                        epochs = 200,
                        train_method= 'random',
                        target_accuracy=0.95,
                        get_alignment_every=10,  
                        validate_every_epoch=None, 
                        base_path='.',
                        lambda_kao=0.01,
                        lambda_co=0.1,
                        clusters=4
                      )

[[0.4762362539768219, 0.5011184215545654], [0.45654547214508057, 0.4528428316116333]]


TypeError: len() of unsized object

In [ ]:
agent.fit_kernel(training_data, training_labels)

tensor([[0.6499, 0.4061],
        [0.6437, 0.8583],
        [0.1163, 0.9145],
        [0.0984, 0.1609],
        [0.6499, 0.4061],
        [0.6437, 0.8583],
        [0.1163, 0.9145],
        [0.0984, 0.1609],
        [0.6499, 0.4061],
        [0.6437, 0.8583],
        [0.1163, 0.9145],
        [0.0984, 0.1609],
        [0.6499, 0.4061],
        [0.6437, 0.8583],
        [0.1163, 0.9145],
        [0.0984, 0.1609]], grad_fn=<RepeatBackward0>)
16
tensor([[0.5892, 0.1535],
        [0.4210, 0.1447],
        [0.8965, 0.1625],
        [0.4231, 0.6479],
        [0.5892, 0.1535],
        [0.4210, 0.1447],
        [0.8965, 0.1625],
        [0.4231, 0.6479],
        [0.5892, 0.1535],
        [0.4210, 0.1447],
        [0.8965, 0.1625],
        [0.4231, 0.6479],
        [0.5892, 0.1535],
        [0.4210, 0.1447],
        [0.8965, 0.1625],
        [0.4231, 0.6479]], grad_fn=<RepeatBackward0>)
32
tensor([[0.8823, 0.9132],
        [0.6437, 0.8583],
        [0.8792, 0.3802],
        [0.8774, 0.6715],
  

KeyboardInterrupt: 